# Introductory Lab for EPG Machine Learning Project

## Part 1: Firebird Setup

To get started on working on the EPG, you'll first need to make sure that you are correctly setup to work on the Firebird cluster. If you haven't already, make sure to follow Jason's instructions for [creating a Firebird account](https://swatkb.atlassian.net/wiki/x/CIAADw) and [logging into Firebird](https://swatkb.atlassian.net/wiki/x/BYCnCQ).

My preferred method for working on Firebird is to connect to it as a [remote host through VSCode](https://code.visualstudio.com/docs/remote/ssh). As described in the Firebird documentation, this will connect you the Firebird login node. It's fine to write code, and do small tasks on here, however YOU SHOULD NOT RUN COMPUTATIONALLY INTENSIVE TASKS ON THE LOGIN NODE. If you do so, they risk getting killed as the login node is shared by all users and needs to remain available. Instead running the actual machine learning should be done through Slurm, as discussed in the next part.

If you prefer to work with an interactive Jupyter noteboook, the easiest way is to open an [interactive desktop](https://swatkb.atlassian.net/wiki/spaces/ACADTECH/pages/162234377/Open+OnDemand#Interactive-Desktop) on a firebird compute node (same as what Slurm will give you) and run jupyter from there. You'll likely want to run Jupyter through uv as discussed [here](https://docs.astral.sh/uv/guides/integration/jupyter/). 

Once you have an appropriate way to connect to Firebird, you should clone this repository into your home directory.

`git clone https://github.com/mespence/hmc-epg-project.git`

You'll then want to checkout this specific branch

`git checkout SwarthmoreTeam`

You can find this file as `machine-learning/IntroLab.ipynb`. You'll likely want to switch over to working on this notebook on Firebird at this point. 

## Part 2: Running the existing code with UV and slurm

This codebase is setup to use [uv](https://docs.astral.sh/uv/) for managing Python packages and environments. Your Firebird account won't have uv by default, but you can install it following the Linux instructions [here](https://docs.astral.sh/uv/getting-started/installation/). I'd recommend reading the documentation on [uv projects](https://docs.astral.sh/uv/guides/projects/) to get a sense how it works.

Once you have uv install, you should try running `uv sync` from this directory or a subdirectory, which will download the nessecary packages. We'll use `uv run ...` to run experiments in this project, however this should either be done within a slurm script or from an interactive desktop session and not directly on the login node.

At this point you should have all the software installed need to run the machine learning code written by the previous teams. First though, check that you have access to the shared data folder: `/data/labs/hopelab/epg`. The mosquito data that we'll start with is in the subdirectory: `tarsalis_data_clean`. Try viewing one of the files with `cat /data/labs/hopelab/epg/tarsalis_data_clean/cxthand1sep2021no1.csv` or through the VSCode file explorer.

Now you should try a test run of the ML code, starting with the random forest model on the mosquito data. First read over [Jason's instructions](https://swatkb.atlassian.net/wiki/spaces/ACADTECH/pages/200212482/Slurm#Batch) for submitting scripts to run on the cluster. There is an example script to submit: `mosquito/slurm.sh`.

Take a look at this script before using it to understand the structure of the code, you should be able to answer:

- How does this code specify the location of the data?
- How does it specify the model to use?
- Where will you be able to find the output?
- What resources is it requesting?

Now try submitting the script to slurm using the `sbatch` command. Once it's running you should be able to check the following:

- What is the JobID the run was assigned?
- Using the job id, find the output file and determine what node the job was assigned to
- Finally take a look at the output images generated by the run. You should be able to identify the performance of the model from the rendered [confusion matrix](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)


## Part 3: Analyzing the data with Pandas and Matplotlib

Before you start modifying the machine learning code itself, you should get a feeling for what the data looks like and how to visualize it. Take a look at the images generated by the script you ran before that plotted the data. The code for this is in the `model_eval.py` file.

- In a jupyter notebook or a script, try plotting some snippets of data as a lineplot using Pandas and Matplotlib or your plotting library of choice. You should use the mosquito data in `/data/labs/hopelab/epg/tarsalis_data_clean`. 
    - The [pandas documentation](https://pandas.pydata.org/docs/user_guide/visualization.html) has some guides for plotting data. 
    - You'll probably find that the files are too long to plot in their entirety, so try plotting 5-10s snippets of the data. You could consider representing the time column of the data as a [pandas timedelta index](https://pandas.pydata.org/docs/user_guide/timedeltas.html), although this isn't nessecary.
    - You should try to replicate or borrow the code from `model_eval.py` to visualize the labels along with the data.
    - Try comparing plotting the "pre-rect" measurements and the "post-rect" measurements to understand how rectification affects the sequence.

Another useful thing to do will be to be able to separate out and visualize different labels. Take a look at the `DataImport` class in `model_eval`, has methods such as `simple_probe_finder` which separate out "probes", in other words, sequences of labels that do not include the "non-probing" (NP) state.

- See if you can modify this probe splitting code in order to plot several examples of each label type present in the mosquito data. This will be helpful for understanding how the entomologists defined the different patterns that make up these types.
- It will also be helpful to analyze what fraction of the data belongs to each label. Again the [pandas documentation](https://pandas.pydata.org/docs/user_guide/10min.html) may be helpful here. You will also relevant code in the `unet.py` file. 

Make sure to save these plots and put them into a report.

## Part 4: Modifying random forests with overlapping windows

Now that you have some understanding of the data and have been able to run the code for training and evaluating, its time to make some modifications to the machine learning parts of the code. For the first part it will be easiest to focus on the random forest model, which is implemented in the `rf.py` file. If you look at the implementations of the different models used: `rf.py`, `unet.py`, `transformer.py` and `tcn.py`, you'll see that they have a `Model` class with 4 common methods: `train`, `predict`, `save` and `load`. This is the standardized inferface used by the code in `model_eval.py`, making it easy to swap in new approaches to the evaluation process. 

In the random forest code, the `train` method trains a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) object from the sklearn library. Because random forests are not natively designed for timeseries data, we need to first split each probe into fixed sized chunks that we can classify. There is a helper method `transform_data` that does this.

The `transform_data` method actually does more than just divide the sequence into chunks, it also turns the original signal for the chunk into a set of features that can be used for classification. In particular it takes the mean, standard deviation, resistance, voltage, current and a few features extracted from a [fast fourier transfor](https://docs.scipy.org/doc/scipy/tutorial/fft.html) together as the features that represent the chunk. The options for creating chunks as well as for the classifier itself are controlled by several properties defined in the `__init__` method of the model.

One thing that could be limiting the performance of the random forest classifier is this "chunking" process, as it means that each 1-3s window can only be assigned a single label, which means it might miss the exact boundaries of a class. There is a trade-off between how granualar the predictions can be (short window/chunk size) and how much contextual information the classifier has for making the prediction (long window/chunk size). One way around this problem is potentially to use overlapping features. For example we could define the chunk size for prediction as 1 second, but compute features using a 3 second window that includes the neighboring chunks. As a next step you should try to implement this for the random forest.

- Modify the `Model` class in `rf.py` to have another property `window_size`, separate from `chunk_seconds` that controls how much of the sequence is used to compute the features for each chunk.
- Modify the `transform_data` method to implement this approach. 

Once we have this new option, we'll need to find the optimal value for it. The code is setup to use a library called Optuna to automatically test out different options in an intelligent way. The `trial` argument to the `__init__` method is an Optuna object that will control choosing these different values. Under `if trial:` you'll see several lines of code that use this object to set values for the different options. 

- You should read through the [Optuna documentation](https://optuna.org/#code_examples) to understand what is code is doing. The `model_eval.py` file has the code that sets up Optuna to find the best values.
- You should add the appropriate code to include your new option in the optuna optimization.
- You should run a slurm job with `model_eval` using the optuna option to find the best value and report the results compared to the original random forest.

Finally, let's try adding a new feature that we'll compute for each chunk. One potentially useful piece of information is the trend of the signal within the window. We can find this simply by fitting a linear regression taking the time as an input and the signal as the output. Scikit-learn's [linear regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class can do this easily. 

- In the `transform_data` loop, add code the that fits a linear regression model for each chunk and appends the `_coef` property as additional features.

## Part 5: Creating a new model

Now that you're hopefull comfortable modifying the code for a model, let's try using a new model! We'll try out a boosting-based classifier, which often performs well in practice.

- First, copy the `rf.py` file and create a new file called `boosting.py`. 

There are several variants of boosting built in to the scikit learn library that we could consider using. We can see the options in the [sklearn ensembles module](https://scikit-learn.org/stable/modules/ensemble.html). The Adaboost or gradient boosting classifers are both likely good choices to try out.

- Modify the train and prediction function of the `Model` class in `boosting.py` to use your choice of boosting classifier. 
- You'll also need to modify the `__init__` function to set appropriate hyperparameters for your choice of classifier. You should set up the optuna code to set an appropriate search space for these options as you did earlier.

Once you have this new model set up you'll need to test it out.

- Run the `model_eval` script again with optuna to find the best performance you can get from a boosting classifier.
- Evaluate how this model compares to the results for the random forest.

## Part 6: Adding new features + classifiers

At this point we're well within the realm of trying out new approaches for this problem. We'll get to the fancy neural-network based models soon enough, but for many real-world problems, careful [feature engineering](https://en.wikipedia.org/wiki/Feature_engineering) can often lead to better performance than sophisticated models. Let's try to extend the the `transform_data` method of the random forest and boosting classifers to have some further new options. 

We'll probably want to start by taking the opportunity to factor out the data preprocessing code into a shared superclass for both models. This will allow us to propagate any improvements to both models and easily add new models using the same data pipeline.

- Create a new file called something like `tabular_model.py` and add a class with the `transform_data` method. Then have both the random forest and boosting models inheret from this class instead of implementing their own versions of the method.

Now we can try out some improvements! I'm not going to tell you exactly what to do for now, but I'll give some suggestions for where to look and let you decide what make sense. I'd recommend dividing up these options to work on in parallel.

- [Wavelet transforms](https://www.cs.toronto.edu/~mangas/teaching/320/slides/CSC320L11.pdf) could be a useful alternative to the FFT features currently being used. [PyWavelets](https://pywavelets.readthedocs.io/en/latest/) has implementations that might be useful.

- More generally [SKTime](https://www.sktime.net/en/stable/) has many options for timeseries feature extration and classification

- You could also try some basic extensions to what we already have, such as breaking each window into subsection to compute statistics or adding more statistics such as percentiles.
- A potentially very powerful approach that does not require deep knowledge of building neural networks would be to leverage a timeseries foundation model. The [MOMENT model](https://arxiv.org/pdf/2402.03885) has a [tutorial](https://github.com/moment-timeseries-foundation-model/moment/blob/main/tutorials/classification.ipynb) for using it for classification.